In [2]:
import pandas
import numpy as np
from keras import layers
from keras import models
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

from keras.utils import plot_model

import pydot


In [3]:
seed = 7 
np.random.seed(seed)

In [4]:
dataframe = pandas.read_csv('iris.csv')
dataset = dataframe.values
X=dataset[:,0:4].astype(float)
Y=dataset[:,4]
X.shape

(149, 4)

In [5]:
encoder= LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
dummy_y = np_utils.to_categorical(encoded_Y)


In [6]:
def baseline_model():
    
    model= models.Sequential()
    model.add(layers.Dense(8, activation='relu',input_shape=(4,)))
    model.add(layers.Dense(3,activation='softmax'))
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
   # plot_model(model, to_file='multilayer_perceptron_graph.png')
    return model


In [7]:
estimator= KerasClassifier(build_fn=baseline_model,epochs=200,batch_size=5,verbose=0)
kfold= KFold(n_splits=10,shuffle=True,random_state=seed)
results= cross_val_score(estimator,X,dummy_y,cv=kfold)
print("Result: %.2f%%(%.2f%%)"%(results.mean()*100,results.std()*100))

Result: 96.62%(4.51%)


Step 5 Tunning making model smaller and larger 

In [8]:
def baseline_model_larger():
    
    model= models.Sequential()
    model.add(layers.Dense(8, activation='relu',input_shape=(4,)))
    model.add(layers.Dense(8,activation='relu'))
    model.add(layers.Dense(8,activation='relu'))
    model.add(layers.Dense(8,activation='relu'))

    model.add(layers.Dense(3,activation='softmax'))
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
   # plot_model(model, to_file='multilayer_perceptron_graph.png')
    return model

In [9]:
estimator= KerasClassifier(build_fn=baseline_model_larger,epochs=200,batch_size=5,verbose=0)
kfold= KFold(n_splits=10,shuffle=True,random_state=seed)
results= cross_val_score(estimator,X,dummy_y,cv=kfold)
print("Result: %.2f%%(%.2f%%)"%(results.mean()*100,results.std()*100))

Result: 96.67%(4.47%)


In [10]:
def baseline_model_smaler():
    
    model= models.Sequential()
    model.add(layers.Dense(2, activation='relu',input_shape=(4,)))
    model.add(layers.Dense(3,activation='softmax'))
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
   # plot_model(model, to_file='multilayer_perceptron_graph.png')
    return model

In [11]:
estimator= KerasClassifier(build_fn=baseline_model_smaler,epochs=200,batch_size=5,verbose=0)
kfold= KFold(n_splits=10,shuffle=True,random_state=seed)
results= cross_val_score(estimator,X,dummy_y,cv=kfold)
print("Result: %.2f%%(%.2f%%)"%(results.mean()*100,results.std()*100))

Result: 74.14%(32.49%)


Step 6 overfit

In [12]:
def baseline_model_overfit():
    
    model= models.Sequential()
    model.add(layers.Dense(16, activation='relu',input_shape=(4,)))
    model.add(layers.Dense(16,activation='relu'))
    model.add(layers.Dense(16,activation='relu'))
    model.add(layers.Dense(16,activation='relu'))

    model.add(layers.Dense(3,activation='softmax'))
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
   # plot_model(model, to_file='multilayer_perceptron_graph.png')
    return model

In [16]:
estimator= KerasClassifier(build_fn=baseline_model_overfit,epochs=400,batch_size=5,verbose=0)
kfold= KFold(n_splits=10,shuffle=True,random_state=seed)
results= cross_val_score(estimator,X,dummy_y,cv=kfold)
print("Result: %.2f%%(%.2f%%)"%(results.mean()*100,results.std()*100))

Result: 96.62%(4.51%)


step 7 tune with optimzer and change no off epochs

In [20]:
def baseline_model_more_tuned():
    
    model= models.Sequential()
    model.add(layers.Dense(4, activation='relu',input_shape=(4,)))
    model.add(layers.Dense(3,activation='softmax'))
    model.compile(optimizer='rmsprop',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
   # plot_model(model, to_file='multilayer_perceptron_graph.png')
    return model

In [21]:
estimator= KerasClassifier(build_fn=baseline_model_more_tuned,epochs=100,batch_size=5,verbose=0)
kfold= KFold(n_splits=10,shuffle=True,random_state=seed)
results= cross_val_score(estimator,X,dummy_y,cv=kfold)
print("Result: %.2f%%(%.2f%%)"%(results.mean()*100,results.std()*100))

Result: 94.67%(4.99%)


Step 8 fucntional api

In [28]:
from keras.layers import Input
from keras.layers import Dense
from keras.models import Model

def create_model_with_functionalAPI():
    
    visible = Input(shape=(4,))
    hidden = Dense(8, activation='relu')(visible)
    output = Dense(3, activation='softmax')(hidden)
    model = Model(inputs=visible,outputs=output)
  
    model.compile(optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy'])
    return model

In [29]:
estimator= KerasClassifier(build_fn=create_model_with_functionalAPI,epochs=100,batch_size=5,verbose=0)
kfold= KFold(n_splits=10,shuffle=True,random_state=seed)
results= cross_val_score(estimator,X,dummy_y,cv=kfold)
print("Result: %.2f%%(%.2f%%)"%(results.mean()*100,results.std()*100))

Result: 98.22%(2.95%)


Step 10: Without sklearn

In [35]:
import numpy as np
k = 10
data = len(X) // k
num_epochs = 200
all_scores = []
val_loss=[]
val_acc=[]
for i in range(k):
    print('processing fold #', i)
    val_data = X[i * data: (i + 1) * data]
    val_targets = dummy_y[i * data: (i + 1) * data]
    
    partial_train_data = np.concatenate([X[:i * data],X[(i + 1) * data:]],axis=0)
    partial_train_targets = np.concatenate([dummy_y [:i * data],dummy_y[(i + 1) * data:]],axis=0)
    
    model = baseline_model()
    model.fit(partial_train_data, partial_train_targets,shuffle=True,epochs=num_epochs, batch_size=5, verbose=0)
    
    results6 = model.evaluate(val_data,val_targets, verbose=0)
    
    val_loss.append(results6[0])
    val_acc.append(results6[1])
    

processing fold # 0
processing fold # 1
processing fold # 2
processing fold # 3
processing fold # 4
processing fold # 5
processing fold # 6
processing fold # 7
processing fold # 8
processing fold # 9


In [37]:
print("Result: %.2f%%(%.2f%%)"%(np.mean(val_acc)*100,np.std(val_acc)))

Result: 95.00%(0.08%)
